In [ ]:
import os 
import json
import pandas as pd
from collections import defaultdict


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 이미지 및 JSON 폴더 경로 설정
training_image_dir = '/content/drive/MyDrive/DCC/dataset/training_image'  # 학습 데이터 이미지가 저장된 디렉터리 경로
validation_image_dir = '/content/drive/MyDrive/DCC/dataset/validation_image' # 검증 데이터 이미지가 저장된 디렉터리 경로
training_label_dir = '/content/drive/MyDrive/DCC/dataset/training_label' # 학습 데이터의 라벨 (JSON 파일) 디렉터리 경로
validation_label_dir = '/content/drive/MyDrive/DCC/dataset/validation_label'# 검증 데이터의 라벨 (JSON 파일) 디렉터리 경로


In [ ]:
# 2-1

# 이미지 파일 이름에서 이미지ID 추출하는 함수
def extract_image_id(filename):
    # 파일 이름을 '_' 문자로 분할하여 리스트로 만듬
    # ex) 'W_95569_19_normcore_W.jpg' -> ['W', '95569', '19', 'normcore', 'W.jpg']
    return filename.split('_')[1]  
 # 분할된 리스트의 두 번째 요소(ID)를 반환

In [ ]:
# 이미지ID 수집
def collect_image_ids(image_dir):
    image_ids = set() # 중복되지 않는 이미지 ID를 저장할 set을 생성
    for filename in os.listdir(image_dir): #지정된 디렉터리의 파일을 for문을 통해 순회
        if filename.endswith('.jpg'):#파일 이름이 '.jpg'로 끝나는 경우에만 처리
            image_id = extract_image_id(filename) # 파일명에서 이미지 ID를 추출
            image_ids.add(image_id)  # 추출된 이미지 ID를 set에 추가
    return image_ids  # 모든 이미지 ID가 저장된 set을 반환


In [ ]:
# 이미지ID 수집
training_image_ids = collect_image_ids(training_image_dir) 

validation_image_ids = collect_image_ids(validation_image_dir)
#디렉터리에서 이미지 ID를 수집하여 training_image_ids / validation_image_ids에 저장

In [ ]:
# JSON 파일에서 성별, 스타일 정보를 추출
def extract_gender_style_from_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        gender = data['item']['gender']
        style = data['item']['style']
        return gender, style


In [ ]:
# 유효한 JSON 파일만 처리하여 통계 계산
def calculate_statistics(image_ids, label_dir):
    stats = defaultdict(lambda: defaultdict(int))

    for filename in os.listdir(label_dir):
        if filename.endswith('.json'):
            image_id = extract_image_id(filename)
            if image_id in image_ids:  # 이미지ID가 실제로 존재하는지 확인
                json_path = os.path.join(label_dir, filename)
                gender, style = extract_gender_style_from_json(json_path)
                stats[gender][style] += 1  # 성별 및 스타일에 따른 이미지 수 증가

    return stats


In [ ]:
# 통계 계산
training_stats = calculate_statistics(training_image_ids, training_label_dir)
#라벨 디렉터리에서 유요한 값 처리 후 calculate_statistics 에서 처리한 성별 및 스타일 통계를 저장
validation_stats = calculate_statistics(validation_image_ids, validation_label_dir)
#트레이닝 에서 했던것과 동일하게 벨리데이션에서도 적용


In [ ]:
# 통계 결과를 표 형식으로 변환하고 내림차순으로 정렬
def generate_table(stats, dataset_type):
    rows = []
    for gender, styles in stats.items():
        for style, count in styles.items():
            rows.append([gender, style, count])# 각 성별, 스타일, 이미지 수를 하나의 행으로 추가

    df = pd.DataFrame(rows, columns=['성별', '스타일', '이미지 수'])

    # 성별 기준으로 먼저 정렬한 후, 이미지 수 기준으로 내림차순 정렬
    df_sorted = df.sort_values(by=['성별', '이미지 수'], ascending=[True, False]).reset_index(drop=True)

    return df_sorted.head(1000)

In [ ]:
# 정렬된 표 작성
generate_table(training_stats, "Training")

In [ ]:
generate_table(validation_stats, "Validation")

In [ ]:
# 2-2

# JSON 파일에서 응답자 스타일 선호/비선호 정보를 수집하는 함수
def collect_preference_data(image_ids, label_dir):
    preference_data = defaultdict(lambda: {'선호': [], '비선호': []})

    for filename in os.listdir(label_dir):
        if filename.endswith('.json'):
            image_id = extract_image_id(filename)
            if image_id in image_ids:  # 이미지ID가 유효한 경우에만 처리
                json_path = os.path.join(label_dir, filename)
                with open(json_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)
                    respondent_id = data['user']['R_id']
                    style = data['item']['style']
                    img_filename = data['item']['imgName']  # 이미지 파일명
                    preference = data['item']['survey']['Q5']  # 1: 비선호, 2: 선호

                    if preference == 1:
                        preference_data[respondent_id]['비선호'].append(img_filename)
                    elif preference == 2:
                        preference_data[respondent_id]['선호'].append(img_filename)

    return preference_data

In [ ]:
# 선호/비선호 데이터 수집
training_preference_data = collect_preference_data(training_image_ids, training_label_dir)
validation_preference_data = collect_preference_data(validation_image_ids, validation_label_dir)

In [ ]:
# Pandas 출력 옵션 설정 (더 많은 행과 열을 출력하도록 설정)
pd.set_option('display.max_rows', None)  # 모든 행 출력
pd.set_option('display.max_columns', None)  # 모든 열을 출력
pd.set_option('display.width', 1000)  # 출력 가로폭을 확장

In [ ]:
# 열의 최대 너비 설정 (문자열을 일정한 길이로 자르고 공백을 추가)
def format_string(value, max_len=35):
    if len(value) > max_len:
        return value[:max_len-3] + '...'  # 텍스트가 길면 자르기
    else:
        return value.ljust(max_len)  # 텍스트가 짧으면 오른쪽에 공백 추가

In [ ]:
# 각 응답자에 대한 스타일 선호/비선호 파일명을 셀에 나누어 표시하는 함수
def prepare_rows_for_respondent(respondent_id, training_preferences, validation_preferences):
    max_len = max(len(training_preferences['선호']), len(training_preferences['비선호']),
                  len(validation_preferences['선호']), len(validation_preferences['비선호']))

    rows = []
    for i in range(max_len):
        row = [
            respondent_id if i == 0 else '',  # 첫 번째 행에만 응답자 ID 출력
            format_string(training_preferences['선호'][i]) if i < len(training_preferences['선호']) else format_string(''),
            format_string(training_preferences['비선호'][i]) if i < len(training_preferences['비선호']) else format_string(''),
            format_string(validation_preferences['선호'][i]) if i < len(validation_preferences['선호']) else format_string(''),
            format_string(validation_preferences['비선호'][i]) if i < len(validation_preferences['비선호']) else format_string('')
        ]
        rows.append(row)

    return rows

In [ ]:
# 표 경계를 위한 함수
def print_row_separator():
    print("+------------+-------------------------------------+-------------------------------------+-------------------------------------+-------------------------------------+")

In [ ]:
# 상위 100명에 대해 Training과 Validation 스타일 선호/비선호 정보를 나누어 표 작성
def generate_combined_preference_table_separated(training_data, validation_data, top_respondents):
    rows = []
    for respondent_id in top_respondents:
        training_preferences = training_data.get(respondent_id, {'선호': [], '비선호': []})
        validation_preferences = validation_data.get(respondent_id, {'선호': [], '비선호': []})

        # 응답자별로 스타일 정보를 여러 행에 나누어 추가
        rows += prepare_rows_for_respondent(respondent_id, training_preferences, validation_preferences)

    # DataFrame 생성
    df = pd.DataFrame(rows, columns=['응답자 ID', 'Training 스타일 선호', 'Training 스타일 비선호', 'Validation 스타일 선호', 'Validation 스타일 비선호'])

    print("\n--- 상위 100명에 대한 Training 및 Validation 스타일 선호/비선호 통계 ---\n")

    # 표 헤더 출력
    print_row_separator()
    print(f"| {'응답자 ID':^7} |{'Training 스타일 선호':^32} | {'Training 스타일 비선호':^30} | {'Validation 스타일 선호':^31} | {'Validation 스타일 비선호':^30} |")
    print_row_separator()

    # 표 내용 출력
    previous_respondent = None
    for idx, row in df.iterrows():
         # 응답자 ID가 바뀔 때만 구분선 출력
        if row['응답자 ID'] != previous_respondent:
            if previous_respondent is not None:  # 첫 번째 행은 제외
                print_row_separator()  # 응답자 변경시 구분선

        respondent_id = row['응답자 ID'] if row['응답자 ID'] != previous_respondent else ""
        print(f"| {respondent_id:^10} | {row['Training 스타일 선호']:<35} | {row['Training 스타일 비선호']:<35} | {row['Validation 스타일 선호']:<35} | {row['Validation 스타일 비선호']:<35} |")

        previous_respondent = row['응답자 ID']

    # 마지막 줄 경계 출력
    print_row_separator()

    return df

In [ ]:
# Training과 Validation에서 공통된 응답자ID 찾기
common_respondents = set(training_preference_data.keys()) & set(validation_preference_data.keys())

In [ ]:
# 응답자별 총 응답수 기준으로 상위 100명 선택
respondent_response_count = {
    respondent_id: len(training_preference_data[respondent_id]['선호']) + len(training_preference_data[respondent_id]['비선호']) +
                   len(validation_preference_data[respondent_id]['선호']) + len(validation_preference_data[respondent_id]['비선호'])
    for respondent_id in common_respondents
}

In [ ]:

# 총 응답수 기준 상위 100명의 응답자 선택
top_100_respondents = sorted(respondent_response_count, key=respondent_response_count.get, reverse=True)[:100]

# 표 작성 (상위 100명의 응답자에 대한 데이터)
top_100_preference_df = generate_combined_preference_table_separated(training_preference_data, validation_preference_data, top_100_respondents)

top_100_preference_df.head(1000000000000)

# CSV 파일로 저장하여 전체 데이터 확인 가능
top_100_preference_df.to_csv('top_100_preference_with_totals.csv', index=False)
